# 2일차 2교시 워드임베딩 - 실습
한선아

Contents
1. Word2vec
2. FastText
3. Glove
4. Visualization

## Part 0 실습 준비하기

### 코랩 파일 업로드

* `wiki_small800.txt` : 한국어 위키 텍스트
* `NanumBarunpenB.otf` : 시각화 시 한글이 깨지지 않도록 사용하는 폰트
* `kor_ws353` : 단어 유사도(word similarity) 테스트 데이터
* `kor_analogy_semantic` : 의미론적 단어 유추 테스트 데이터

### 라이브러리 import

In [ ]:
# 사용할 라이브러리들을 import 합니다.
import gensim 
from gensim.models import Word2Vec, FastText
from gensim.models import KeyedVectors
import pandas as pd

* Gensim 라이브러리-[What is Gensim?](https://radimrehurek.com/gensim/intro.html)
  * 텍스트 데이터를, 의미를 가진 벡터로 표현하기 위해 필요한 기능들을 지원하는 라이브러리입니다.    
  Gensim을 통해 Word2Vec, FastText와 같은 알고리즘을 사용할 수 있습니다.

### 코퍼스 데이터 가져오기

In [ ]:
# 사용할 코퍼스의 경로
path = '/content/wiki_small800.txt'

In [ ]:
# 가져온 데이터를 확인해보겠습니다.
df = pd.read_csv(path, encoding="utf-8", header=None)
df

In [ ]:
# 첫 번째 문서
df[0][0]

In [ ]:
# 다섯 번째 문서
df[0][4]

## Part 1 워드 임베딩 모델

### 1) Word2Vec

* 2013년 구글 연구팀이 발표한 기법으로, 가장 널리 쓰이고 있는 단어 임베딩 모델입니다.
* Word2Vec 모델의 학습방법 2가지
    * **CBOW 모델** : 문맥 단어들을 가지고 타깃 단어 하나를 맞추는 과정에서 학습됩니다.
    * **Skip-gram 모델** : 타깃 단어를 가지고 주변 문맥 단어가 무엇일지 예측하는 과정에서 학습됩니다.
<img src="https://blog.kakaocdn.net/dn/Czgg5/btqEttXkz91/LK5RqukCujicrxQ2kRWt0k/img.png" height=300>
* [Gensim API Reference - Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)

#### Word2Vec 모델 학습

In [ ]:
# 모델에 입력값으로 사용할 수 있도록 적합한 객체로 바꿉니다.
corpus = gensim.models.word2vec.Text8Corpus(path)

print(list(corpus)[0][:100])

In [ ]:
# 가져온 데이터(corpus)로 Word2Vec 모델을 학습시킵니다.
model = Word2Vec(sentences=corpus, size=100, window=3, min_count=5, sg=1)

* `sentences` : 단어리스트 (a list of lists of tokens)
* `size` : 임베딩 벡터의 차원
* `window` : 중심단어를 예측하기 위해서 앞, 뒤로 볼 단어의 개수(범위)
* `min_count` : 빈도수가 얼마나 작으면 제외(ignore)할 지 결정
* `sg` : 훈련 알고리즘. 1 - skip-gram, 0 - CBOW

#### Word2Vec 임베딩 결과 확인

In [ ]:
# 모델의 학습 결과로 얻은 워드 벡터들을 저장해둡니다.
word2vec_vectors = model.wv
word2vec_vectors.save("word2vec.wordvectors")

del model

In [ ]:
# '컴퓨터'에 해당하는 밀집 벡터를 확인해보겠습니다. 
word2vec_vectors['컴퓨터'] # 코퍼스에 존재하는 단어

#### Word2Vec 임베딩 평가

In [ ]:
# 저장해두었던 워드벡터들을 불러옵니다.
word2vec_vectors = KeyedVectors.load("word2vec.wordvectors")

##### 가장 유사한 단어 출력

In [ ]:
# '컴퓨터'와 가장 유사한 단어 10개 출력
word2vec_vectors.most_similar('컴퓨터', topn=10)

In [ ]:
# 코퍼스에 존재하지 않는 단어는 오류가 납니다.
word2vec_vectors.most_similar('전산언어학겨울학교', topn=10)

##### 두 단어의 유사도 계산

In [ ]:
# '컴퓨터'와 '유럽'의 유사도
word2vec_vectors.similarity('컴퓨터', '유럽')

In [ ]:
# '컴퓨터'와 '웹'의 유사도
word2vec_vectors.similarity('컴퓨터', '웹')

##### 가장 유사하지 않은 단어 출력

In [ ]:
# '일본', '중국', '미국' 중 가장 유사하지 않은 단어 출력
word2vec_vectors.doesnt_match(['일본','중국', '미국'])

##### 단어벡터의 연산

In [ ]:
# 왕 + 여성 - 남성 = ???
word2vec_vectors.most_similar(positive=['왕', '여성'], negative=['남성'], topn=5)

### 2) FastText

* 페이스북에서 개발해 공개한 단어 임베딩 기법
* word2vec과 기본적으로 동일하나, 각 단어를 문자(Character) 단위 n-gram으로 표현합니다.
* FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주합니다. 내부 단어, 즉 서브워드(subword)를 고려하여 학습합니다.
* 코퍼스에 없는 모르는 단어(Out Of Vocabulary)에도 대처할 수 있다는 장점이 있습니다.
* [Gensim API Reference - FastText](https://radimrehurek.com/gensim/models/fasttext.html)

#### FastText 모델 학습

In [ ]:
# 가져온 데이터(corpus)로 FastText 모델을 학습시킵니다.
model = FastText(sentences=corpus, size=100, window=3, min_count=5, sg=1)

* `sentences` : 단어리스트 (a list of lists of tokens)
* `size` : 임베딩 벡터의 차원
* `window` : 중심단어를 예측하기 위해서 앞, 뒤로 볼 문자의 개수(범위)
* `min_count` : 빈도수가 얼마나 작으면 제외(ignore)할 지 결정
* `sg` : 훈련 알고리즘. 1 - skip-gram, 0 - CBOW

#### FastText 임베딩 결과 확인

In [ ]:
# 모델의 학습 결과로 얻은 워드벡터들을 저장해둡니다.
FastText_vectors = model.wv
FastText_vectors.save("fasttext.wordvectors")

del model

In [ ]:
#  '컴퓨터'에 해당하는 밀집 벡터를 확인해보겠습니다.
FastText_vectors['컴퓨터']

#### FastText 임베딩 평가하기

In [ ]:
# 저장해두었던 워드벡터들을 불러옵니다.
FastText_vectors = KeyedVectors.load("fasttext.wordvectors")

##### 가장 유사한 단어 출력

In [ ]:
FastText_vectors.most_similar('컴퓨터', topn=10)

In [ ]:
# 코퍼스에 없는 단어도 유사도를 계산할 수 있습니다.
FastText_vectors.most_similar('전산언어학겨울학교', topn=10)

##### 두 단어의 유사도 분석

In [ ]:
FastText_vectors.similarity('미국', '영국')

In [ ]:
FastText_vectors.similarity('미국', '함수')

### 3) Glove
* 미국 스탠포드대학교연구팀에서 개발한 단어 임베딩 기법
* 유사도 계산의 성능이 좋으면서도, 윈도우 내의 로컬문맥(local context)만 학습하지 않고 전체의 통계정보를 반영하고자 고안된 기법입니다.
* 단어-문맥 행렬(동시 등장 행렬, co-occurrence matrix)을 사용합니다.
 * 오늘 뭐 먹고 싶어
 * 나는 오늘 연어 먹고 싶어
 * 나는 어제 연어 먹었어


  | 카운트 | 오늘 | 뭐  | 먹고 | 싶어 | 나는 | 연어 | 어제 | 먹었어 |
  | ------ | ---- | --- | ---- | ---- | ---- | ---- | ---- | ------ |
  | 오늘   |  0    |  1  |   0   |  0    |  1   | 1    |  0    |    0    |
  | 뭐     |  1   |  0   |   1  |    0  |  0    |   0   |    0 |   0     |
  | 먹고   |   0   |  1  |  0    |   2  |  0    |   1  |   0   |    0    |
  | 싶어   |  0    |   0  |    2 |    0  |  0    |   0   |    0  |   0     |
  | 나는   |    1 |   0  |  0    |  0    |    0  |   0   |  1   |     0   |
  | 연어   | 1    |  0   |  1   |   0   | 0     |   0   |   1  |    1   |
  | 어제   |  0    |   0  |   0   |     0 |    1 |   1  |     0 |   0     |
  | 먹었어 |   0   |  0   |  0    |   0   |    0  | 1    |  0    |  0      |

* [glove-python GIthub](https://github.com/maciejkula/glove-python)




In [ ]:
# Python Glove를 사용하기 위해 라이브러리를 설치합니다.
! pip install glove-python-binary

In [ ]:
# 라이브러리를 import 합니다.
from glove import Glove, Corpus

#### 단어-문맥 행렬 만들기

In [ ]:
# 코퍼스 객체를 선언합니다. 
data = gensim.models.word2vec.Text8Corpus(path)
corpus = Corpus()

In [ ]:
# 코퍼스 어휘 사전과 동시등장행렬(단어-문맥행렬)을 생성합니다.
corpus.fit(data, window=3)

# 코퍼스 안의 딕셔너리 크기와, 연어의 개수를 출력합니다.
print('Dict size : %s' % len(corpus.dictionary))
print('Collocations: %s' % corpus.matrix.nnz)       # nnz : non-zero element

# 코퍼스를 corpus.model로 저장합니다.
corpus.save('corpus.model')

#### Glove 모델 학습하기

In [ ]:
# Glove 모델을 선언합니다.
glove = Glove(no_components=100, learning_rate=0.05)

# Glove 모델을 학습합니다.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)

* `no_components` : 임베딩벡터의 차원
* `learning_rate` : 학습률 - 모델을 업데이트할 때 사용하는 보폭
* `epoch` : 에포크- 학습 횟수

In [ ]:
# Glove 모델을 사용하기 위해서는 모델에 해당 단어 사전 올려주어야 합니다.
glove.add_dictionary(corpus.dictionary)

# 모델을 저장합니다.
glove.save('glove.model')

#### Glove 모델 평가

In [ ]:
glove = Glove.load('glove.model')
glove.most_similar('언어', number=10)

## Part 2 Visualization

* word2vec 임베딩 결과를 시각화합니다.
* 임베딩 벡터의 차원을 100차원으로 했기 때문에, 시각화를 위해 우리가 이해할 수 있는 2차원, 3차원의 저차원으로 축소해야합니다.
*  t-SNE(t-distributed Stochastic Neighbor Embedding)
  * 차원 축소 시에, 단어간의 거리가 가깝고 먼 정도를 최대한 보존하기 위한 방법론
  * 원 공간의 데이터 확률 분포와 축소된 공간의 분포 사이의 차이를 최소화하는 방향으로 벡터 공간을 조금씩 바꿔나갑니다.

* [t-SNE 개념과 사용법 참고](https://gaussian37.github.io/ml-concept-t_sne/)

In [ ]:
import re
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from collections import Counter

from sklearn.manifold import TSNE
import matplotlib.font_manager as fm
import matplotlib as mpl
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

In [ ]:
# 시각화에서 한글이 깨지지 않도록 폰트를 올려줍니다.
path_nanum = "/content/NanumBarunpenB.otf"
prop = fm.FontProperties(fname=path_nanum)

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False 

In [ ]:
# 간단한 전처리를 합니다.
FILTERS = "([~,!?\"':.;|~)^(])"
CHANGE_FILTER = re.compile(FILTERS)

EXP = "[1234567890\-]"
CHANGE_EXP = re.compile(EXP)

ENG = "[a-zA-Z]"
CHANGE_ENG = re.compile(ENG)

#df = pd.read_csv(path, header=None)
words = []
for line in df[0]:
    line = CHANGE_FILTER.sub("", line)
    line = CHANGE_EXP.sub("", line)
    line = CHANGE_ENG.sub("", line)
    token = word_tokenize(line)
    words+=token

In [ ]:
# word2vec 임베딩 결과를 불러옵니다.
word2vec_wordvectors = KeyedVectors.load('word2vec.wordvectors')

# 가장 빈도수가 높은 800개의 단어를 추출합니다.
freq_list = Counter(words).most_common(800)
vocab = [i[0] for i in freq_list if len(i[0])>1]

# 해당 단어들에 해당하는 임베딩 벡터
X = word2vec_wordvectors[vocab]

### 1) tsne 2차원 축소

In [ ]:
#2차원으로 축소하는 t-SNE 모델을 생성합니다.
tsne_2d_model = TSNE(perplexity=15,n_components=2, n_iter=3600, random_state=0)

* `perplexity` : 학습에 영향을 주는 점들의 개수를 조절합니다.    

    보통 5~50사이의 값을 사용하며, 값이 작을 수록 global structure 보다 local structure에 더 집중합니다.
* `n_compontnets` : 임베딩 공간의 차원
* `n_iter` : 최적화를 위한 최대 반복 횟수입니다. 최소한 250 이상은 되어야 합니다.
* `random_state` : 난수(random number) 생성 알고리즘에서 사용하는 seed(씨앗)을 설정합니다.  
* 파라미터에 대한 자세한 설명은 [여기](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)를 참조하세요.

In [ ]:
def tsne_2d(title, tsne, X):
    # 100차원에서 2차원으로 임베딩 결과를 축소합니다.
    X_tsne = tsne.fit_transform(X[:300,:])
    # 각 단어별 x좌표와 y좌표를 Dataframe으로 저장합니다.
    df = pd.DataFrame(X_tsne, index=vocab[:300], columns=['x', 'y'])
    
    # 그래프를 생성하고 출력합니다.
    %matplotlib inline              
    fig = plt.figure()              # 그래프 생성
    fig.set_size_inches(20, 10)     # 그래프 사이즈 설정
    ax = fig.add_subplot(1, 1, 1)   # 2D 축 생성
    ax.scatter(df["x"], df["y"])    # 각 좌표에 점 표시
    for word, pos in list(df.iterrows()):
        ax.annotate(word, pos, fontsize=12, fontproperties=prop) # 단어 주석
    plt.title(title)                # 제목 표시
    plt.show()                      # 그래프출력

In [ ]:
tsne_2d('Visualizing Embeddings using t-SNE', tsne_2d_model, X)

### 2) tsne 3차원 축소

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

In [ ]:
# 3차원으로 축소하는 t-SNE 모델을 만들어줍니다.
tsne_3d_model = TSNE(perplexity=15, n_components=3, n_iter=3500, random_state=0)

In [ ]:
def tsne_3d(title, tsne, X, a=1):
    # 100차원에서 3차원으로 임베딩 결과를 축소합니다.
    X_tsne = tsne.fit_transform(X)
    # 각 단어별 x,y,z좌표를 Dataframe으로 저장합니다.
    df = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y', 'z'])

    # 그래프를 생성하고 출력합니다.
    %matplotlib inline
    fig = plt.figure()                      # 그래프 이미지 생성
    ax = fig.add_subplot(projection='3d')   # 3D 축 생성
    ax.scatter(df["x"], df["y"], df["z"], c='crimson', alpha=a) # 각 좌표에 점 표시
    plt.title(title)        # 제목 표시
    plt.show()              # 그래프 출력

In [ ]:
tsne_3d('Visualizing Embeddings using t-SNE', tsne_3d_model, X, a=0.2)

#### 👀[참고] 특정 단어의 3차원 시각화

In [ ]:
def tsne_3d_sample(title, words, tsne, a=1):    
    # 100차원에서 3차원으로 임베딩 결과를 축소합니다.
    X_tsne = tsne.fit_transform(X)
    # 각 단어별 x,y,z좌표를 Dataframe으로 저장합니다.
    df = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y', 'z'])
    # 그래프를 생성하고 출력합니다.
    %matplotlib inline
    fig = plt.figure()                      # 그래프 이미지 생성
    ax = fig.add_subplot(projection='3d')   # 3D 축 생성
    for word in words:                      # 샘플 단어 좌표 및 주석 표시
        x = df["x"][word]
        y = df["y"][word]
        z = df["z"][word]
        ax.scatter(x, y, z, c='crimson', alpha=a)               
        ax.text(x, y, z, word, fontsize=10, zorder=1, fontproperties=prop)    
    plt.title(title)    # 제목 표시
    plt.show()          # 그래프 출력

In [ ]:
sample_words=['지구','독일', '영국', '미국']
tsne_3d_sample('Visualizing Embeddings using t-SNE', sample_words, tsne_3d_model, a=1)

## 👀 [참고] Word Embedding Test
* 자연어 단어 간 품사적, 의미론적 관계가 임베딩에 얼마나 잘 녹아 있는지 정량적으로 평가해봅니다.

* 단어 유사도 평가(Word similarity test)
  * 일련의 단어쌍을 미리 구성한 후에 사람이 평가한 점수와 단어 벡터 간 코사인 유사도 사이의 상관관계(correlation)를 계산해 단어 임베딩의 품질을 평가하는 방법

* 단어 유추 평가(Word analogy test)
  * 단어벡터간 계산을 통해 질의에 대한 정답을 도출해낼 수 있는지 평가
*  데이터와 코드 참고 :[이동준님의 github](https://github.com/dongjun-Lee/kor2vec)

In [ ]:
import seaborn as sns
import scipy.stats as st

### 1) 단어 유사도 평가

In [ ]:
#평가 데이터의 사람이 평가한 유사도 점수와, 모델의 임베딩벡터 쌍 간 코사인 유사도의 상관관계를 반환합니다.
def word_sim_test(test_fname, wordvectors):
        actual_sim_list, pred_sim_list = [], []
        missed = 0
        with open(test_fname, 'r') as pairs:
            for pair in pairs:
                w1, w2, actual_sim = pair.strip().split(",")
                try:
                    pred_sim = wordvectors.similarity(w1, w2)         # 모델의 임베딩 벡터 쌍 간 코사인 유사도
                    actual_sim_list.append(float(actual_sim))         # 사람이 평가한 유사도 점수수
                    pred_sim_list.append(pred_sim)
                except KeyError:
                    missed += 1
                    
        spearman, _ = st.spearmanr(actual_sim_list, pred_sim_list) # 스피어만 상관계수
        pearson, _ = st.pearsonr(actual_sim_list, pred_sim_list)   # 피어슨 상관계수
        return spearman, pearson, missed

In [ ]:
# 테스트셋의 경로입니다.
test_fname = "/content/kor_ws353.csv"

In [ ]:
# 위에서 학습했던 Word2Vec과 FastText의 임베딩 결과(워드벡터)를 불러옵니다.
word2vec_wordvectors = KeyedVectors.load('word2vec.wordvectors')
fasttext_wordvectors = KeyedVectors.load('fasttext.wordvectors')

In [ ]:
# 두 모델의 단어 유사도 평가를 수행합니다.
word2vec_test = word_sim_test(test_fname, word2vec_wordvectors)
fasttext_test = word_sim_test(test_fname, fasttext_wordvectors)

In [ ]:
# 상관계수가 1에 가까울 수록 둘 사이의 상관관계가 강하다는 뜻입니다.
# (spearman, pearson, missed)
print(word2vec_test)
print(fasttext_test)

In [ ]:
# 유사도 평가 결과를 데이터프레임으로 출력합니다.
df = pd.DataFrame({"Word Embedding": ["Word2Vec","Word2Vec","FastText","FastText"]})
df["Criterion"] = ["spearman", "pearson"]*2
df["score"] = list(word2vec_test[:-1] + fasttext_test[:-1])
df

In [ ]:
# 유사도 평가 결과를 bar plot으로 출력합니다.
sns.barplot(x="Word Embedding", y="score", hue="Criterion", data=df)
plt.title("Word Similarity Test Result")

### 2) 단어 유추 평가

In [ ]:
# 벡터 간 계산의 결과가 의미론적 유추에서의 정답을 도출해낼 수 있는지 평가합니다.
def word_analogy_test(test_fname, wordvectors):
        correct, total, missed = 0, 0, 0

        with open(test_fname, 'r', encoding='utf-8') as f:
            for line in f:
                if line.startswith("#") or len(line) <= 1: continue
                words = line.strip().split(" ")

                try:
                    predicted_answer = [i[0] for i in wordvectors.most_similar(positive=[words[0], words[2]], negative= [words[1]], topn=30)]
                    # print(words[0] + " - " + words[1] + " + " + words[2])
                    # print("correct answer:", words[3])
                    # print("predicted answers:", predicted_answer[0])
                    # print("")
                    if words[-1] in predicted_answer: correct += 1
                except:
                    missed += 1
                
                total += 1
        print(wordvectors)
        print("# of correct answer:", correct, ", # of data:", total, ", # of errors:", missed)
        print()
        return correct/(total-missed) # 맞춘 개수 / 처리 데이터 수

In [ ]:
test_fname = "/content/kor_analogy_semantic.txt"

In [ ]:
# 두 모델의 단어 유추 평가를 수행합니다.
word2vec_test = word_analogy_test(test_fname, word2vec_wordvectors)
fasttext_test = word_analogy_test(test_fname, fasttext_wordvectors)

In [ ]:
# 유추 평가 결과를 데이터프레임으로 출력합니다.
df2 = pd.DataFrame({"Word Embedding": ["Word2Vec","FastText"]})
df2["score"] = [word2vec_test, fasttext_test]
df2

In [ ]:
# 유사도 평가 결과를 bar plot으로 출력합니다.
sns.barplot(x="Word Embedding", y="score", data=df2)
plt.title("Word Analogy Test Result")